# --- Day 9: Smoke Basin ---
https://adventofcode.com/2021/day/9

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in.

Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example (not shown in notebook), there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [64]:
import pprint
import time
def readHeightmap():
    with open('heightmap.txt') as file:
        return file.read()
def readHeightmapTest():
    with open('heightmapTest.txt') as file:
        return file.read()
    
print(readHeightmapTest())
for i in range(6): #Just takes up more space so the output is smaller. Nothing important
    print("")

2199943210
3987894921
9856789892
8767896789
9899965678








In [3]:
#formatting input
heightmap = readHeightmap().split('\n') #Splits by new line
for i in range(len(heightmap)):
    heightmap[i] = [*heightmap[i]] #Splits each line by character
for i in range(len(heightmap)):
    for j in range(len(heightmap[i])):
        heightmap[i][j] = int(heightmap[i][j]) #Casts each number in each line as an integer

allRiskLevels = []
for row in range(len(heightmap)):
    for col in range(len(heightmap[i])):
        height = heightmap[row][col]
        up = None
        down = None
        left = None
        right = None
        if abs(row-1) == row-1: #Gets the value of the height above heightmap[row][col] if it exists else None
            up = heightmap[row-1][col]
        if row+1 < len(heightmap): #Gets the value of the height below heightmap[row][col] if it exists else None
            down = heightmap[row+1][col]
        if abs(col-1) == col-1: #Gets the value of the height left of heightmap[row][col] if it exists else None
            left = heightmap[row][col-1]
        if col+1 < len(heightmap[row]): #Gets the value of the height right of heightmap[row][col] if it exists else None
            right = heightmap[row][col+1]
        
        if up != None and down != None and left != None and right != None:
            if (up > height) and (down > height) and (left > height) and (right > height):
                allRiskLevels.append(height+1)
        if up == None and down != None and left == None and right != None: #up == left == None
            if down > height and right > height:
                allRiskLevels.append(height+1)
        if up == None and down != None and left != None and right == None: #up == right == None
            if down > height and left > height:
                allRiskLevels.append(height+1)
        if up == None and down != None and left != None and right != None: #up == None
            if down > height and left > height and right > height:
                allRiskLevels.append(height+1)
        if up != None and down == None and left == None and right != None: #down == left == None
            if up > height and right > height:
                allRiskLevels.append(height+1)
        if up != None and down == None and left != None and right == None: #down == right == None
            if up > height and left > height:
                allRiskLevels.append(height+1)
        if up != None and down == None and left != None and right != None: #down == None
            if up > height and left > height and right > height:
                allRiskLevels.append(height+1)
        if up != None and down != None and left == None and right != None: #left == None
            if up > height and down > height and right > height:
                allRiskLevels.append(height+1)
        if up != None and down != None and left != None and right == None: #right == None
            if up > height and down > height and left > height:
                allRiskLevels.append(height+1)
                
        #print(f"Row: {row} | Val: {heightmap[row][col]} | Up: {up} | Down: {down} | Left: {left} | Right: {right}")
print(f"Sum of all risk levels: {sum(allRiskLevels)}")

Sum of all risk levels: 516


# --- Part Two ---
Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

In [67]:
#formatting input
def resetHeightmap():
    heightmap = readHeightmap().split('\n') #Splits by new line
    for i in range(len(heightmap)):
        heightmap[i] = [*heightmap[i]] #Splits each line by character
    for i in range(len(heightmap)):
        for j in range(len(heightmap[i])):
            heightmap[i][j] = int(heightmap[i][j]) #Casts each number in each line as an integer
    return heightmap
        
def getSurroundingHeights(heightmap, row, col):
    up = None
    down = None
    left = None
    right = None
    if abs(row-1) == row-1: #Gets the value of the height above heightmap[row][col] if it exists else None
        up = heightmap[row-1][col]
    if row+1 < len(heightmap): #Gets the value of the height below heightmap[row][col] if it exists else None
        down = heightmap[row+1][col]
    if abs(col-1) == col-1: #Gets the value of the height left of heightmap[row][col] if it exists else None
        left = heightmap[row][col-1]
    if col+1 < len(heightmap[row]): #Gets the value of the height right of heightmap[row][col] if it exists else None
        right = heightmap[row][col+1]
    return up, down, left, right

def recurseBasin(heightmap, row, col):
    up, down, left, right = getSurroundingHeights(heightmap, row, col)
    heightmap[row][col] = "B"
    
    if up and up!=9 and up!="B":
        heightmap = recurseBasin(heightmap, row-1, col)
    if down and down!=9 and down!="B":
        heightmap = recurseBasin(heightmap, row+1, col)
    if left and left!=9 and left!="B":
        heightmap = recurseBasin(heightmap, row, col-1)
    if right and right!=9 and right!="B":
        heightmap = recurseBasin(heightmap, row, col+1)
    return heightmap

start = time.time()

heightmap = resetHeightmap()
bCount = 0
basins = []
#This nested for loop creates a grid like map using rows and columns
for row in range(len(heightmap)): #Loops through the rows in heightmap as row
    for col in range(len(heightmap[row])): #Loops through the rows in heightmap[row] as col
        height = heightmap[row][col] #Sets the value of the current point to the variable height
        up, down, left, right = getSurroundingHeights(heightmap, row, col)
        
        if up != None and down != None and left != None and right != None:
            if (up > height) and (down > height) and (left > height) and (right > height):
                heightmap = recurseBasin(heightmap, row, col)
        if up == None and down != None and left == None and right != None: #up == left == None
            if down > height and right > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up == None and down != None and left != None and right == None: #up == right == None
            if down > height and left > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up == None and down != None and left != None and right != None: #up == None
            if down > height and left > height and right > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up != None and down == None and left == None and right != None: #down == left == None
            if up > height and right > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up != None and down == None and left != None and right == None: #down == right == None
            if up > height and left > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up != None and down == None and left != None and right != None: #down == None
            if up > height and left > height and right > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up != None and down != None and left == None and right != None: #left == None
            if up > height and down > height and right > height:
                heightmap = recurseBasin(heightmap, row, col)
        if up != None and down != None and left != None and right == None: #right == None
            if up > height and down > height and left > height:
                heightmap = recurseBasin(heightmap, row, col)
        if "B" in heightmap[row]:
            #pprint.pprint(heightmap)
            bCount = 0
            for i in range(len(heightmap)):
                for j in range(len(heightmap[i])):
                    if heightmap[i][j] == "B":
                        bCount+=1
            basins.append(bCount)
                
        heightmap = resetHeightmap()
        
basins = list(sorted(basins))
print(f"Three biggest basins multiplied together: {basins[-1]*basins[-2]*basins[-3]}")
print(f"List of all basin sizes: {basins}")
print(f"Time to complete: {time.time() - start} seconds")

Three biggest basins multiplied together: 1023660
List of all basin sizes: [2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 24, 24, 25, 25, 25, 25, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 32, 33, 33, 34, 34, 34, 34, 36, 37, 37, 38, 38, 39, 39, 39, 39, 39, 39, 40, 41, 41, 42, 42, 42, 44, 44, 45, 45, 46, 46, 47, 48, 48, 48, 48, 49, 49, 49, 51, 51, 51, 51, 52, 53, 53, 53, 54, 54, 55, 56, 56, 57, 58, 58, 58, 59, 60, 61, 61, 62, 62, 62, 63, 63, 63, 66, 66, 67, 68, 68, 69, 70, 71, 72, 72, 72, 72, 75, 75, 75, 78, 78, 78, 79, 79, 81, 84, 88, 88, 88, 89, 89, 94, 99, 110]
Time to complete: 21.073798656463623 seconds


# Part 2 Improved
This is pretty much a more optimized version of the last code where instead of counting all values == 'B' in the entire list every time a basin is occured, recurseBasin will now return a list of coordinates recursed. There is an issue that occurs where duplicates are in the list, so when taking the length of the list it's converted to a set because it does not allow for duplicates.

In [71]:
#formatting input
def resetHeightmap():
    heightmap = readHeightmap().split('\n') #Splits by new line
    for i in range(len(heightmap)):
        heightmap[i] = [*heightmap[i]] #Splits each line by character
    for i in range(len(heightmap)):
        for j in range(len(heightmap[i])):
            heightmap[i][j] = int(heightmap[i][j]) #Casts each number in each line as an integer
    return heightmap
        
def getSurroundingHeights(heightmap, row, col):
    """This gets the values above, below, and to the sides of heightmap[row][col] and is None if it doesn't exist"""
    up = None
    down = None
    left = None
    right = None
    if abs(row-1) == row-1: #Gets the value of the height above heightmap[row][col] if it exists else None
        up = heightmap[row-1][col]
    if row+1 < len(heightmap): #Gets the value of the height below heightmap[row][col] if it exists else None
        down = heightmap[row+1][col]
    if abs(col-1) == col-1: #Gets the value of the height left of heightmap[row][col] if it exists else None
        left = heightmap[row][col-1]
    if col+1 < len(heightmap[row]): #Gets the value of the height right of heightmap[row][col] if it exists else None
        right = heightmap[row][col+1]
    return up, down, left, right

def recurseBasin(heightmap, row, col, basin=None):
    """This function uses recursion to traverse the area of a basin using 9 as a boundry. It returns the heightmap with 
    points inside the basin labeled as 'B' and a list of points the function has traveled through. Sometimes duplicates are
    in the list when done recursing, but that is dealt with later by turning it into a set before taking the length"""
    up, down, left, right = getSurroundingHeights(heightmap, row, col)
    heightmap[row][col] = "B" #Sets the point to "B" in order to keep track of already recursed points
    #This keeps track of the points being recursed. For some reason duplicates show up, but that's dealt with later
    if basin:
        basin.append(f'{row, col}')
    else:
        basin = [f'{row, col}']
        
    #Checks each direction to make sure it's not 9 or already recursed ("B"). If not 9 or "B" the recursion will continue
    if up and up!=9 and up!="B":
        heightmap, basin = recurseBasin(heightmap, row-1, col, basin)
    if down and down!=9 and down!="B":
        heightmap, basin = recurseBasin(heightmap, row+1, col, basin)
    if left and left!=9 and left!="B":
        heightmap, basin = recurseBasin(heightmap, row, col-1, basin)
    if right and right!=9 and right!="B":
        heightmap, basin = recurseBasin(heightmap, row, col+1, basin)
    return heightmap, basin

heightmap = resetHeightmap()
basins = []
#This nested for loop creates a grid like map using rows and columns
for row in range(len(heightmap)): #Loops through the rows in heightmap as row
    for col in range(len(heightmap[row])): #Loops through the rows in heightmap[row] as col
        height = heightmap[row][col] #Sets the value of the current point to the variable height
        up, down, left, right = getSurroundingHeights(heightmap, row, col)
        
        if up != None and down != None and left != None and right != None: #If height is surrounded by elements
            if (up > height) and (down > height) and (left > height) and (right > height):
                heightmap, basinSize = recurseBasin(heightmap, row, col) #Recurses the basin to get all points
        elif up == None and down != None and left != None and right != None: #If height is at the top row
            if down > height and left > height and right > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up != None and down == None and left != None and right != None: #If height is at the bottom row
            if up > height and left > height and right > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up != None and down != None and left == None and right != None: #If height is on the left side of heightmap
            if up > height and down > height and right > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up != None and down != None and left != None and right == None: #If height is on the right side of heightmap
            if up > height and down > height and left > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up == None and down != None and left == None and right != None: #If height is at the top left corner
            if down > height and right > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up == None and down != None and left != None and right == None: #If height is at the top right corner
            if down > height and left > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up != None and down == None and left == None and right != None: #If height is at the bottom left corner
            if up > height and right > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        elif up != None and down == None and left != None and right == None: #If height is at the bottom right corner
            if up > height and left > height: #If height is less than all it's surrounding values
                heightmap, basinSize = recurseBasin(heightmap, row, col)
        if "B" in heightmap[row]: #If there was a basin
            basins.append(len(set(basinSize))) #This gets the length of the list of points in the basin (no duplicates)
        heightmap = resetHeightmap() #Resets the heightmap to avoid further issues
        
basins = list(sorted(basins))
print(f"Three biggest basins multiplied together: {basins[-1]*basins[-2]*basins[-3]}")
print(f"List of all basin sizes: {basins}")

Three biggest basins multiplied together: 1023660
List of all basin sizes: [2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 21, 21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 24, 24, 25, 25, 25, 25, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 32, 33, 33, 34, 34, 34, 34, 36, 37, 37, 38, 38, 39, 39, 39, 39, 39, 39, 40, 41, 41, 42, 42, 42, 44, 44, 45, 45, 46, 46, 47, 48, 48, 48, 48, 49, 49, 49, 51, 51, 51, 51, 52, 53, 53, 53, 54, 54, 55, 56, 56, 57, 58, 58, 58, 59, 60, 61, 61, 62, 62, 62, 63, 63, 63, 66, 66, 67, 68, 68, 69, 70, 71, 72, 72, 72, 72, 75, 75, 75, 78, 78, 78, 79, 79, 81, 84, 88, 88, 88, 89, 89, 94, 99, 110]
